# M2 actuator movies
Craig Lage - 23-May-24 \
The M2 mirror has 72 axial actuators and 6 tangential actuators. 

In [ ]:
import sys, time, os, yaml, warnings
import shlex, subprocess
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from PIL import Image, ImageOps
from scipy.ndimage import rotate
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


## Set up the necessary subroutines

In [ ]:
def actuatorLayout(ax, yaml_file):
    # Get the data from the yaml file
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    axials = np.array(locations['locAct_axial'])
    xs = axials[:,0]
    ys = axials[:,1]
    ax.set_xlabel("Y position (m)")
    ax.set_ylabel("X position (m)")
    ax.set_title("M2 Actuator positions and type", fontsize=18)

    ax.scatter(ys, xs, marker='o', s=200, color='gold', label='Axial')
    deltaX = -0.05
    deltaY = -0.07
    for i in range(len(xs)):
        ax.text(ys[i]+deltaY, xs[i]+deltaX, f"{i+1}", fontsize=9)

    # Now plot tangential actuator locations
    Rtan = locations['radiusActTangent'] # Radius in meters
    thetas = locations['locAct_tangent']
    width = 0.2
    height = 0.9
    poly = [[-height/2.0, 0],[height/2.0, 0], \
                     [height/2.0, width], [-height/2.0, width]]
    scale = 2.5
    for i, theta in enumerate(thetas):
        theta *= np.pi / 180.0
        rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                       [np.sin(theta), np.cos(theta)]]).transpose()
        rot_poly = []
        for point in poly:
            rot_poly.append(np.dot(rot_matrix, point))
        rot_poly = np.array(rot_poly)
        x = Rtan * np.cos(theta)
        y = Rtan * np.sin(theta)
        center = [y + np.mean(rot_poly[:,0]), x + np.mean(rot_poly[:,1])]
        xy = np.array([y,x]) + rot_poly
        deltaX = -0.04
        deltaY = -0.02
        ax.text(center[0]+deltaY, center[1]+deltaX,  f"{i+1}", fontsize=9)
        if i == 0:
            ax.add_patch(Polygon(
                xy=xy, linewidth=1, color='coral', \
                fill=True, label='Tangential'))
        else:
            ax.add_patch(Polygon(
                xy=xy, linewidth=1, color='coral', fill=True))
    ax.legend(loc='lower left', fontsize=9)
    ax.axis('equal')   
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    return
    
def plotAxialForces(df, yaml_file, ax, zmin, zmax, time):
    # Get the data from the yaml file
    index = df.index.searchsorted(time)
    print(f"Axial index = {index}")
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    axials = np.array(locations['locAct_axial'])
    xs = axials[:,0]
    ys = axials[:,1]
    zs = []
    for i in range(len(xs)):
        name=f"measured{i}"
        zs.append(df.iloc[index][name])

    ax.set_xlabel("Y position (m)")
    ax.set_ylabel("X position (m)")
    ax.set_title("M2 actuator forces (N)", fontsize=18)

    im = ax.scatter(ys, xs, marker='o', c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=200, label="Axial")
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r')
    ax.text(-1.5, 3.5, "Axial force colorbar", color='black', rotation=90)
    return


def plotTangentForces(df, yaml_file, ax, zmax, time):
    index = df.index.searchsorted(time)
    print(f"Tangent index = {index}")
    # Get the data from the yaml file
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    # Now plot tangential actuator locations
    Rtan = locations['radiusActTangent'] # Radius in meters
    thetas = locations['locAct_tangent']
    width = 0.2
    height = 0.9
    poly = [[-height/2.0, 0],[height/2.0, 0], \
                     [height/2.0, width], [-height/2.0, width]]
    scale = 2.5
    for i, theta in enumerate(thetas):
        theta *= np.pi / 180.0
        rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                       [np.sin(theta), np.cos(theta)]]).transpose()
        rot_poly = []
        for point in poly:
            rot_poly.append(np.dot(rot_matrix, point))
        rot_poly = np.array(rot_poly)
        x = Rtan * np.cos(theta)
        y = Rtan * np.sin(theta)
        xy = np.array([y,x]) + rot_poly

        if i == 0:
            ax.add_patch(Polygon(
                xy= xy, linewidth=1, color='coral', \
                fill=True, label='Tangent'))
        else:
            ax.add_patch(Polygon(
                xy=xy, linewidth=1, color='coral', fill=True))

        name=f"measured{i}"
        force =  - df.iloc[index][name]
        length = 0.5 * force / zmax
        dx = length * np.sin(theta)
        dy =  - length * np.cos(theta)
        box_center = np.array([x + np.mean(rot_poly[:,1]), y + np.mean(rot_poly[:,0])])
        arrow_center = box_center + np.array([width * np.cos(theta), width * np.sin(theta)])
        arrow_start = arrow_center - np.array([dx / 2.0, dy / 2.0])       
        ax.arrow(arrow_start[1], arrow_start[0], dy, dx, width=0.02, length_includes_head=True, color='black')
    legend_force = 1000.0
    legend_length = 0.5 * legend_force / zmax
    xs = -scale * 0.9
    ys = scale * 0.9
    dx = legend_length
    dy = 0
    ax.arrow(xs, ys, dx, dy, width=0.02, length_includes_head=True, color='black')
    ax.text(xs + dx * 1.05, ys,  f"{legend_force:.1f} N", color='black')
    ax.legend(bbox_to_anchor=(1.45, 0.1),  fontsize=9)
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    return

def plotTelescope(df, simonyi_file, time):
    # Get the elevation angle at the given time
    index = df.index.searchsorted(time)
    print(f"Elevation index = {index}")
    angle = 90.0 - df['actualPosition'].iloc[index]
    tel = Image.open(simonyi_file)   
    tel = ImageOps.expand(tel, border=100, fill=(255,255,255)) 
    rot_tel = rotate(tel, angle, reshape=False)
    #The OffsetBox is a simple container artist.
    #The child artists are meant to be drawn at a relative position to its #parent.
    imagebox = OffsetImage(rot_tel, zoom = 0.15)
    #Annotation box for solar pv logo
    #Container for the imagebox referring to a specific position *xy*.
    ab = AnnotationBbox(imagebox, (1.6, 1.0), xycoords='axes fraction', box_alignment=(1.1, 1), frameon = False)
    ax.add_artist(ab)
    ax.text(3.7, 0.6, "Telescope\nElevation", color='black')
    return
    

# Make a plot of the actuator layout.

In [ ]:
yaml_file = '../cell_geom/cell_geom.yaml'
fig, ax = plt.subplots(1,1,figsize=(5,5))
actuatorLayout(ax, yaml_file)
plt.savefig("../cell_geom/Actuator_Layout.png")

## Define the times and options

In [ ]:
# Times to start looking at encoder values
start = Time("2024-04-24T00:00:00", scale='utc')
end = Time("2024-04-24T02:00:00", scale='utc')
start = Time("2024-01-10T00:00:00", scale='utc')
end = Time("2024-01-10T04:00:00", scale='utc')

yaml_file = '../cell_geom/cell_geom.yaml'
simonyi_file = '../cell_geom/Simonyi.png'

axial_zmax = 250
tangent_zmax = 2500
# The following allows you to plot only every nth data point
# If this value is 1, a frame will be made for every data point
# Of course, this takes longer
# If this value is 50, it will make a frame every second
frameN = 100

## Now generate the frames
### This will take some time

In [ ]:
warnings.filterwarnings("ignore") # Stop annoying matplotlib warning

client = EfdClient('usdf_efd')
start = Time("2024-04-24T00:00:00", scale='utc')
end = Time("2024-04-24T02:00:00", scale='utc')
start = Time("2024-01-10T00:00:00", scale='utc')
end = Time("2024-01-10T03:00:00", scale='utc')

timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
dirName = f"/home/c/cslage/u/MTM2/movies/movie_{timestamp}"
%mkdir -p {dirName}
movieName = f"m2_movie_{timestamp}.mp4"

axial_forces = await client.select_time_series("lsst.sal.MTM2.axialForce", "*", \
                                             start, end)
tangent_forces = await client.select_time_series("lsst.sal.MTM2.tangentForce", "*", \
                                             start, end)
elevation = await client.select_time_series("lsst.sal.MTMount.elevation", \
                                            ['actualPosition', 'private_efdStamp'], \
                                             start, end)
# Add a layout with numbering
fig = plt.figure(figsize=(8,5))
ax = fig.add_axes([0.10, 0.10, 0.5, 0.8], aspect='equal')
actuatorLayout(ax, yaml_file)
plt.savefig(f"{dirName}/Frame_{0:05d}.png")
plt.clf()
# Now build the individual frames
for n in range(0, int(len(elevation)/10), frameN):
    time = elevation.index[n]
    ax = fig.add_axes([0.10, 0.10, 0.5, 0.8], aspect='equal')
    plotAxialForces(axial_forces, yaml_file, ax, 0, axial_zmax, time)
    plotTangentForces(tangent_forces, yaml_file, ax, tangent_zmax, time)
    plotTelescope(elevation, simonyi_file, time)
    plt.savefig(f"{dirName}/Frame_{(n + 1):05d}.png")
    plt.clf()
    nFrames = int(n / frameN)
    if nFrames%100 == 0:
        print(f"{nFrames} frames done")



## Now build the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()